## Model 1

### Install and Import Necessary libraries

In [1]:
!pip install -U albumentations
!pip install opencv-python==4.5.4.60
!pip install timm==0.6.2.dev0

In [2]:

import torch
torch.cuda.empty_cache()

In [3]:
import os
import albumentations
from albumentations.pytorch import ToTensorV2
import pandas as pd
import numpy as np
import gc
import timm

In [5]:
from fastai.vision.all import *

### Seed For Reproducibility

In [6]:
def random_seed(seed_value, use_cuda): 
    np.random.seed(seed_value)
 #cpu vars
    torch.manual_seed(seed_value) 
# cpu  vars
    random.seed(seed_value)
 # Python 
    if use_cuda: 
        torch.cuda.manual_seed(seed_value) 
        torch.cuda.manual_seed_all(seed_value) 
# gpu vars
        torch.backends.cudnn.deterministic = True 
 #needed
        torch.backends.cudnn.benchmark = False 
#Remember to use num_workers=0 when creating the DataBunch.



In [7]:
random_seed(2022,True)

In [8]:
#Start by connecting gdrive into the google colab

from google.colab import drive

drive.mount('/content/gdrive')

### Load the Data

In [9]:
path = Path('../input/ricediseasedetection/Rice disease detector/Rice disease detector')

df = pd.read_csv('../input/ricediseasedetection/normaltrain.csv')


df['Image_id'] = "../input/ricediseasedetection/Rice disease detector/Rice disease detector/Images/" + df['Image_id']

df.head()

In [15]:
df.Label.value_counts()

In [16]:
data_path =Path('../input/ricediseasedetection/Rice disease detector/Rice disease detector')

### Augmentations using Albumentations

In [17]:
class AlbumentationsTransform (RandTransform):
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [19]:
def get_train_aug(): return albumentations.Compose([
            albumentations.Resize(224, 224), #Extra tip, use size that's suitable for the efficentNetwork you are using.
            albumentations.Rotate(limit=(-90, 90)),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.RandomGamma(),
            albumentations.RandomGridShuffle(p=0.5)
            ], p=1.)



def get_valid_aug(): return albumentations.Compose([
            albumentations.Resize(224, 224),
            ], p=1.0)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

In [20]:
df.head()

### DataLoader

In [21]:
blocks = (ImageBlock, MultiCategoryBlock)
splitter = TrainTestSplitter(0.1, stratify=df["Label"],random_state=42,shuffle=True)
block = DataBlock(blocks = blocks,
                 get_x=ColReader(0),
                 get_y=ColReader(1,label_delim=' '),
                 splitter = splitter,
                 item_tfms = item_tfms,
                 batch_tfms = batch_tfms)

In [23]:
dls = block.dataloaders(df, bs=16)

In [24]:
dls.show_batch(figsize=(12,12))

In [25]:
from fastai.metrics import accuracy_multi

In [26]:
# timm.list_models('efficient*')

### Training and Ealuation

In [27]:
learn = vision_learner(dls, 'swin_large_patch4_window7_224',  metrics=[accuracy_multi], cbs=[GradientAccumulation(),SaveModelCallback(),EarlyStoppingCallback(patience=3)])
learn.to_fp16()

In [28]:
# del variables
gc.collect()

In [29]:
learn.lr_find()

In [30]:
learn.fine_tune(15,0.009120108559727669)

Better model found at epoch 13 with valid_loss value: 0.011384588666260242

In [38]:
learn.to_fp32()

### Inference

In [39]:
sample_df = pd.read_csv(data_path/'SampleSubmission.csv')
sample_df.head()

In [40]:
sample_copy = sample_df.copy()



sample_df['Image_id'] = "../input/ricediseasedetection/Rice disease detector/Rice disease detector/Images/" + sample_df['Image_id']

sample_df.head()

In [41]:
test_dl = learn.dls.test_dl(sample_df)

In [42]:
preds, _ = learn.tta(dl=test_dl, n=15)
preds[:,1]

In [43]:
k=[]
for col in sample_df.columns: 
  k.append(col) # creating list of the label

k

In [44]:
import os
submission = pd.DataFrame()
submission["Image_id"] = sample_copy["Image_id"]
for i, c in enumerate(learn.dls.vocab):
  print(c)
  submission[c] = preds[:,i]
submission.head()

In [45]:
submission.to_csv('./swin_final.csv', index=False)

In [46]:
learn.recorder.final_record 